#            *Data Science Project: Toronto City*

## In this project, our goal is extract a dataset from Wikipedia about Toronto city and make a analysis with information fitted and revealing

In [99]:
#Let's start to import some libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas.io.html import read_html
print('libraries imported')

libraries imported


In [100]:
# url from wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [101]:
#extraction
wiki = read_html(url, attrs={'class':'wikitable'})
print('ewxtratc {num} wiki'.format(num=len('wiki')))

ewxtratc 4 wiki


In [102]:
wiki = wiki[0]

In [103]:
#Dropping 'Not assigned'
wiki = wiki.set_index("Borough")
wiki = wiki.drop("Not assigned", axis=0)

In [105]:
# Apply groupby funtion and display DataFrame
wiki = wiki.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list)
wiki = wiki.sample(frac=1).reset_index()
wiki['Neighbourhood'] = wiki['Neighbourhood'].str.join(' , ')
wiki.rename(columns = {'Postcode':'Postal Code'}, inplace = True) 
wiki.head()


,Postal Code,Borough,Neighbourhood
0,M1N,Scarborough,"Birch Cliff , Cliffside West"
1,M5S,Downtown Toronto,"Harbord , University of Toronto"
2,M9L,North York,Humber Summit
3,M3K,North York,"CFB Toronto , Downsview East"
4,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade


In [106]:
wiki['Neighbourhood']=wiki['Neighbourhood'].replace('Not assigned', wiki['Borough'])

In [107]:
# How many rows and columns has our DataFrame? 
print('the DataFrame has ' + str(wiki.shape[0]) + ' rows and ' + str(wiki.shape[1]) + ' columns')

the DataFrame has 103 rows and 3 columns


### The next step is add longitude and latitude values, to star let's import some libraries and get data from a link.

In [108]:
import pandas as pd
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c35661ee0ed14ac7a11485a8abe7f253 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Rt2RqqVwZU9V5sFPwKPYg9dLZgoBusQJRJYb3YAYHgnp',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_c35661ee0ed14ac7a11485a8abe7f253.get_object(Bucket='torontoanalysis-donotdelete-pr-pjyzgj7f23e6fu',Key='Geospatial_Coordinates (1).csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### The Join: In this step, let's use merge funtion to bring together the latitude and logitude values to the original dataframe

In [109]:
df = pd.merge(wiki, df_data_1, on ='Postal Code')
df.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848
1,M5S,Downtown Toronto,"Harbord , University of Toronto",43.662696,-79.400049
2,M9L,North York,Humber Summit,43.756303,-79.565963
3,M3K,North York,"CFB Toronto , Downsview East",43.737473,-79.464763
4,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846


In [110]:
df.shape

(103, 5)

### Use geopy library to get the latitude and longitude values of Toronto


In [111]:
from geopy.geocoders import Nominatim

In [112]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Display the Toronto Map

In [113]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

# All requested packages already installed.



In [114]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

In [115]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Credentials

In [116]:
CLIENT_ID = 'P0KE0ETW5ZCYI5F0OWHVADZ5DB0COYRJXRTNYXAYWOHECEIA' # your Foursquare ID
CLIENT_SECRET = 'B4NMCAKJT1KBQOTDEOHKROLPMARQPUYIR3E2UEUBUEZ5XA5O' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P0KE0ETW5ZCYI5F0OWHVADZ5DB0COYRJXRTNYXAYWOHECEIA
CLIENT_SECRET:B4NMCAKJT1KBQOTDEOHKROLPMARQPUYIR3E2UEUBUEZ5XA5O


### Top 50 venues in Toronto

In [117]:
df_latitude = latitude # neighborhood latitude value
df_longitude = longitude # neighborhood longitude value

neighborhood_name = 'Toronto'

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               df_latitude, 
                                                               df_longitude))
LIMIT = 50 

radius = 700

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df_latitude, 
    df_longitude, 
    radius, 
    LIMIT)
url


Latitude and longitude values of Toronto are 43.653963, -79.387207.


'https://api.foursquare.com/v2/venues/explore?&client_id=P0KE0ETW5ZCYI5F0OWHVADZ5DB0COYRJXRTNYXAYWOHECEIA&client_secret=B4NMCAKJT1KBQOTDEOHKROLPMARQPUYIR3E2UEUBUEZ5XA5O&v=20180605&ll=43.653963,-79.387207&radius=700&limit=50'

In [118]:
results = requests.get(url).json()

{'meta': {'code': 200, 'requestId': '5e5db37c47b43d0028b88eb5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 224,
  'suggestedBounds': {'ne': {'lat': 43.660263006300006,
    'lng': -79.37851584317704},
   'sw': {'lat': 43.64766299369999, 'lng': -79.39589815682297}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
 

In [119]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [120]:
import json
from pandas.io.json import json_normalize

In [121]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Karine's,Breakfast Spot,43.653699,-79.390743
3,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
4,Manpuku まんぷく,Japanese Restaurant,43.653612,-79.390613


In [122]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

50 venues were returned by Foursquare.


### Analysis by Neighborhood

In [123]:

Toronto = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

Toronto['Neighborhood'] = df['Neighborhood'] 

fixed_columns = [Toronto.columns[-1]] + list(Toronto.columns[:-1])
Toronto = Toronto[fixed_columns]

Toronto.head()

,Vegetarian / Vegan Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,Chinese Restaurant,Clothing Store,...,Record Shop,Restaurant,Salon / Barbershop,Seafood Restaurant,Smoke Shop,Speakeasy,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,University
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
Toronto.shape

(50, 38)

### Group rows by neighborhood and take the mean of the frequency

In [125]:
toronto_grouped = Toronto.groupby('Neighborhood').mean().reset_index()
toronto_grouped

toronto_grouped.shape

(50, 38)

### Display each neighborhood along with the top 5 most common venues

In [126]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                        venue  freq
0                  University   1.0
1                   Jazz Club   0.0
2          Miscellaneous Shop   0.0
3  Modern European Restaurant   0.0
4                 Opera House   0.0


----Albion Gardens , Beaumond Heights , Humbergate , Jamestown , Mount Olive , Silverstone , South Steeles , Thistletown----
                           venue  freq
0                          Hotel   1.0
1  Vegetarian / Vegan Restaurant   0.0
2                    Record Shop   0.0
3             Miscellaneous Shop   0.0
4     Modern European Restaurant   0.0


----Alderwood , Long Branch----
                           venue  freq
0                      Gastropub   1.0
1  Vegetarian / Vegan Restaurant   0.0
2                    Record Shop   0.0
3             Miscellaneous Shop   0.0
4     Modern European Restaurant   0.0


----Bathurst Manor , Downsview North , Wilson Heights----
                           venue  freq
0                          Plaza   1.0

### Put into Frames

In [127]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Top 5 venues for each neighborhood

In [128]:
import numpy as np
num_top_venues = 5


indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,University,Ice Cream Shop,Greek Restaurant,Gastropub,Furniture / Home Store
1,"Albion Gardens , Beaumond Heights , Humbergate...",Hotel,Ice Cream Shop,Greek Restaurant,Gastropub,Furniture / Home Store
2,"Alderwood , Long Branch",Gastropub,University,Ice Cream Shop,Greek Restaurant,Furniture / Home Store
3,"Bathurst Manor , Downsview North , Wilson Heights",Plaza,University,Clothing Store,Greek Restaurant,Gastropub
4,Bayview Village,Clothing Store,University,Ice Cream Shop,Greek Restaurant,Gastropub


## Cluster Neighborhoods

In [129]:
from sklearn.cluster import KMeans

In [130]:
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

### Dataframe that includes the cluster of the top 5 venues for each neighborhood.

In [131]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.dropna()
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848,0,University,Ice Cream Shop,Greek Restaurant,Gastropub,Furniture / Home Store
1,M5S,Downtown Toronto,"Harbord , University of Toronto",43.662696,-79.400049,0,Sushi Restaurant,University,Chinese Restaurant,Greek Restaurant,Gastropub
2,M9L,North York,Humber Summit,43.756303,-79.565963,0,Breakfast Spot,University,Clothing Store,Greek Restaurant,Gastropub
3,M3K,North York,"CFB Toronto , Downsview East",43.737473,-79.464763,0,Ramen Restaurant,University,Greek Restaurant,Gastropub,Furniture / Home Store
4,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846,0,Japanese Restaurant,Ice Cream Shop,Greek Restaurant,Gastropub,Furniture / Home Store
5,M6K,West Toronto,"Brockton , Exhibition Place , Parkdale Village",43.636847,-79.428191,0,Japanese Restaurant,Ice Cream Shop,Greek Restaurant,Gastropub,Furniture / Home Store
6,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819,0,Bubble Tea Shop,University,Clothing Store,Greek Restaurant,Gastropub
7,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park",43.727929,-79.262029,0,Poke Place,University,Clothing Store,Greek Restaurant,Gastropub
8,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Coffee Shop,University,Ice Cream Shop,Greek Restaurant,Gastropub
9,M3H,North York,"Bathurst Manor , Downsview North , Wilson Heights",43.754328,-79.442259,0,Plaza,University,Clothing Store,Greek Restaurant,Gastropub


### create map

In [132]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Finally the DataFrame for each cluster will be display, this last step will bring us value information about Toronto

In [133]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Scarborough,0,University,Ice Cream Shop,Greek Restaurant,Gastropub,Furniture / Home Store
1,Downtown Toronto,0,Sushi Restaurant,University,Chinese Restaurant,Greek Restaurant,Gastropub
2,North York,0,Breakfast Spot,University,Clothing Store,Greek Restaurant,Gastropub
3,North York,0,Ramen Restaurant,University,Greek Restaurant,Gastropub,Furniture / Home Store
4,Downtown Toronto,0,Japanese Restaurant,Ice Cream Shop,Greek Restaurant,Gastropub,Furniture / Home Store
5,West Toronto,0,Japanese Restaurant,Ice Cream Shop,Greek Restaurant,Gastropub,Furniture / Home Store
6,Mississauga,0,Bubble Tea Shop,University,Clothing Store,Greek Restaurant,Gastropub
7,Scarborough,0,Poke Place,University,Clothing Store,Greek Restaurant,Gastropub
9,North York,0,Plaza,University,Clothing Store,Greek Restaurant,Gastropub
10,Central Toronto,0,Ice Cream Shop,University,Greek Restaurant,Gastropub,Furniture / Home Store


In [134]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
26,Scarborough,1,Restaurant,University,Chinese Restaurant,Greek Restaurant,Gastropub
28,East Toronto,1,Restaurant,University,Chinese Restaurant,Greek Restaurant,Gastropub


In [135]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Central Toronto,2,Coffee Shop,University,Ice Cream Shop,Greek Restaurant,Gastropub
31,West Toronto,2,Coffee Shop,University,Ice Cream Shop,Greek Restaurant,Gastropub
32,West Toronto,2,Coffee Shop,University,Ice Cream Shop,Greek Restaurant,Gastropub
47,Downtown Toronto,2,Coffee Shop,University,Ice Cream Shop,Greek Restaurant,Gastropub


In [136]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,East York,3,Vegetarian / Vegan Restaurant,Ice Cream Shop,Greek Restaurant,Gastropub,Furniture / Home Store
34,Etobicoke,3,Vegetarian / Vegan Restaurant,Ice Cream Shop,Greek Restaurant,Gastropub,Furniture / Home Store
37,Central Toronto,3,Vegetarian / Vegan Restaurant,Ice Cream Shop,Greek Restaurant,Gastropub,Furniture / Home Store
